### Load python packages and functions from osm_clipper

In [ ]:
import os,sys
from osm_clipper import single_country,global_shapefiles,gadm36_planet,gadm36_country,poly_files,planet_osm

### Set dat path

In [ ]:
data_path = os.path.join('..','data')

### Download Planet file 

In [ ]:
planet_osm(data_path)

### Download GADM file

In [ ]:
gadm36_planet(data_path)

### Get a cleaned global shapefile of the countries

In [ ]:
global_shapefiles(data_path)

### Create poly files, necessary for the clipping

In [ ]:
global_shape = os.path.join(data_path,'cleaned_shapes','global_countries.gpkg')
poly_files(data_path,global_shape,regionalized=False)

### Run specified country, based on ISO3 code

In [ ]:
country = 'TZA'

In [ ]:
single_country(country, data_path)